In [34]:
import pandas as pd
input_file = "JasData/ASYMMETRIC_PolEr05_dc5_R1_ls13_100MHz_10Degrees_CQT.dat"

In [35]:
import numpy
import matplotlib.pyplot as plt

# from "Security analysis of quantum key distribution with small block length and its application to quantum space communications

def shannon(Q):
    return -Q*numpy.log2(Q)-(1-Q)*numpy.log2(1-Q); 


def optKeyFrac(p, m, QBER, s = 6, f = 1.18):
    beta,xi, nu = p
    return 1-keyFraction(m, QBER, beta, xi, nu, s, f)

def keyFraction(m,QBER,beta,xi,nu, s = 6,f = 1.18): # m is the initial raw raw key length
    
    
    delta = QBER #QBER
    t = -numpy.log2(10**-(s+2)) #correctness
    r = f*shannon(delta) #leaked information
    epsQKD = 10**(-s)#2**-16 # qkd error 
    #m is the raw raw sifted key
    k = numpy.floor(beta*m)
    n = m-k # that's what's left after paramaeter estimation

    Gamma= 1/(m*(delta+xi)+ 1) +1/(m-m*(delta+xi)+ 1)
    PPElim = numpy.exp(-2*m*k*xi**2/(n+1)) + numpy.exp(-2*Gamma*((n*(nu-xi))**2-1))
    epe = numpy.sqrt(PPElim)
       
    tot = 4*(epsQKD - 2**-t - 2*epe)**2   
    
    l = numpy.log2(tot) - t +n*(1-shannon(delta+nu) - r)    
    epa = 0.5*numpy.sqrt(2**(-n*(1-shannon(delta+nu) - r )  + t + l))
   
    if epsQKD < 2**-t + 2*epe + epa:
        l =  -numpy.inf
        
#    if xi > nu:
#        l =  -numpy.inf
#        
#    if nu>0.5-delta:
#        l =  -numpy.inf
#        
#    if beta > 0.5:
#        l =  -numpy.inf
    return l


def optimize(n, qber, s = 10, f = 1.18, loops = 1, steps =101):
      
    par = [0,0,0]


    bounds= [[0,0.5],[0,0.5-qber],[0,0.5-qber]]


    lMax = -numpy.inf
    for loop in range(loops):
#        if bounds[0][0]<0:bounds[0][0]=0;
#        if bounds[0][0]<0:bounds[1][0]=0;
        arr0 = numpy.linspace(bounds[0][0], bounds[0][1] , steps)
        arr1 = numpy.linspace(bounds[1][0], bounds[1][1] , steps)
        
        for i in range(steps): 
            beta = arr0[i]

            for j in range(steps):
                nu = arr1[j]
#
#                if bounds[2][0] < 0:bounds[2][0] = 0
#                
                if bounds[2][1] > nu:
                    arr2 =  numpy.linspace(bounds[2][0], nu, steps)
                else:
                    arr2 =  numpy.linspace(bounds[2][0], bounds[2][1], steps)
                
                
                for q in range(steps):
                    xi = arr2[q]
                    
                    l = keyFraction(n, qber, beta,xi,nu,  s= s, f=f)
                    if l > lMax:
                        if beta > 0.5 or beta <= 0:
                            print('beta range error')
                            continue
                        if xi > nu or xi<=0:
                            print('xi<>nu comparison error')
                            continue
                        if nu > 0.5-qber or nu<=0:
                            print('nu error')
                            continue
                        lMax = l
                        par = [1*beta,1*xi,1*nu]
                        

        rng = 2
        bounds[0] = [par[0]-rng*(numpy.diff(arr0)[0]), par[0]+rng*(numpy.diff(arr0)[0])]
        bounds[1] = [par[2]-rng*(numpy.diff(arr1)[0]), par[2]+rng*(numpy.diff(arr1)[0])]
        bounds[2] = [par[1]-rng*(numpy.diff(arr2)[0]), par[1]+rng*(numpy.diff(arr2)[0])]

        steps = 11
        
    return lMax, par


In [36]:
df = pd.read_csv(input_file,sep='\t')
df

,Elevation,TotRawKey,NormalisedQBER
0,10,246.873605,0.094364
1,11,263.713753,0.088689
2,12,281.584361,0.083409
3,13,300.497385,0.078506
4,14,320.460998,0.073957
...,...,...,...
76,86,451932.402140,0.018933
77,87,452927.991558,0.018912
78,88,453639.530149,0.018896
79,89,454066.617233,0.018887


In [37]:
l_list = []

for index, row in df.iterrows(): 
    qber = row["NormalisedQBER"]
    n = row["TotRawKey"]
    l,par = optimize(n,qber)
    print (index, l, par)
    
    l_list.append(l)
    


0 -inf [0, 0, 0]
1 -inf [0, 0, 0]
2 -inf [0, 0, 0]
3 -inf [0, 0, 0]
4 -inf [0, 0, 0]
5 -inf [0, 0, 0]
6 -inf [0, 0, 0]
7 -inf [0, 0, 0]
8 -inf [0, 0, 0]
9 -inf [0, 0, 0]
10 -188.05383381676205 [0.495, 0.23693582623867335, 0.44704872875221385]
11 -187.87444782968294 [0.5, 0.2383358053374935, 0.4496901987499877]
12 -189.2138740886801 [0.5, 0.22764971413294025, 0.42952776251498154]
13 -189.78799384529958 [0.5, 0.21674751847181903, 0.40895758202230004]
14 -190.25835505499057 [0.5, 0.21290846348207396, 0.401714082041649]
15 -189.8341613140711 [0.5, 0.20794651195854263, 0.38508613325656044]
16 -189.00333716878575 [0.5, 0.20130790650574015, 0.37279241945507435]
17 -187.86409303876513 [0.5, 0.19455639445293024, 0.3602896193572782]
18 -186.251701948483 [0.5, 0.1877025228415954, 0.34759726452147294]
19 -184.28439391335175 [0.5, 0.1841031458129689, 0.3347329923872162]
20 -181.97637763002487 [0.5, 0.18207075036022216, 0.33103772792767666]
21 -178.46842428292237 [0.5, 0.17484546181168717, 0.3179008

In [38]:
df['FiniteKey'] = l_list

In [39]:
df.to_csv("JasData/FiniteKey_ASYMMETRIC_PolEr05_dc5_R1_ls13_100MHz_10Degrees_CQT.csv")

In [6]:
print ('l',l,'beta',par[0])

l 17315.408645310283 beta 0.185


In [5]:
par

[0.185, 0.0455514774, 0.052358020000000005]